In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layer
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Model
from tensorflow.keras import Model
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [2]:
train = pd.read_csv('./data/criteo_sampled_data.csv')
train.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [3]:
# train.info()

In [4]:
cols = train.columns[1:]

In [5]:
dense_feats = [f for f in cols if f[0] == 'I']
sparse_feats = [f for f in cols if f[0] == 'C']

In [6]:
def process_dense_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna(0)
    for f in feats:
        d[f] = d[f].apply(lambda x: np.log(x+1) if x>-1 else -1)
    return d
data_dense = process_dense_feats(train, dense_feats)
    

In [7]:
def process_sparse_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna('-1')
    for f in feats:
        d[f] = LabelEncoder().fit_transform(d[f])
    return d
data_sparse = process_sparse_feats(train, sparse_feats)

In [8]:
total_data = pd.concat([data_dense, data_sparse], axis=1)
total_data['label'] = train['label']

In [9]:
# 如果你只是想对流经该层的数据做个变换，而这个变换本身没有什么需要学习的参数，那么直接用Lambda Layer是最合适的了

In [10]:
# 获取类别型特征的大小
sparse_feat_config= dict()
for col in sparse_feats:
    sparse_feat_config[col] = total_data[col].nunique()

In [11]:
# 获取类别型特征的大小
dense_feat_config= []
for col in dense_feats:
    dense_feat_config.append(col)

In [12]:
# 构造验证集和训练集
train_data = total_data.loc[:500000-1]
valid_data = total_data.loc[500000:]

train_dense_x = [train_data[f].values for f in dense_feats]#  train_data[dense_feats] 
train_sparse_x = [train_data[f].values for f in sparse_feats] # train_data[sparse_feats] # 
train_label = train_data['label'].values
train_label = tf.cast(train_label, tf.int32)

val_dense_x = [valid_data[f].values for f in dense_feats] # valid_data[dense_feats]   
val_sparse_x = [valid_data[f].values for f in sparse_feats] # valid_data[sparse_feats]
val_label = valid_data['label'].values
val_label = tf.cast(val_label, tf.int32)


In [13]:
# 构造训练集和测试集
def make_data(total_data,idx):
    train_data = total_data.loc[idx,:]
    train_dense_x = [train_data[f].values for f in dense_feats]
    train_sparse_x = [train_data[f].values for f in sparse_feats]
    train_label = train_data['label'].values
    return train_sparse_x,train_dense_x,train_label

# 写法一
继承layer,定义不同功能的层

In [14]:
# 独立层：
# sparse 嵌入层
class Embedding_sparse_layer(tf.keras.layers.Layer):
    def __init__(self,sparse_feat_config, embeding_shape):
        super(Embedding_sparse_layer, self).__init__()
        # l2正则化
        self.reg_1 = tf.keras.regularizers.l2(0.1)
        self.embed_first = {}
        self.sparse_feat_config = sparse_feat_config
        self.embeding_shape = embeding_shape
        self.sparse_feat = list(sparse_feat_config.keys())
        for key, value in self.sparse_feat_config.items():
            self.embed_first[key] = layer.Embedding(value+1,self.embeding_shape, 
                                                    embeddings_regularizer=self.reg_1, 
                                                    name='embed'+key)
    def call(self,x_sparse):
        embed_lookup_first = []
        for i,key in enumerate(self.sparse_feat):

            _embed = self.embed_first[key](x_sparse[i])

            embed_lookup_first.append(_embed)

        return embed_lookup_first
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'sparse_feat_config': self.sparse_feat_config,
            'embeding_shape': self.embeding_shape,
        })
        return config
# t = Embedding_dense(sparse_feat_config,1)
# y = t(inputs_sparse)    

In [23]:
# 独立层：
# dense 嵌入层
class Embedding_dense_layer(tf.keras.layers.Layer):
    def __init__(self, embeding_shape):
        super(Embedding_dense_layer, self).__init__()
        self.embeding_shape = embeding_shape
        self.embeds = []
 
    def build(self,input_shape):
#         print('input',input_shape)
        for _ in enumerate(input_shape):
            embed = tf.Variable(
                        initial_value = tf.random_normal_initializer()(shape=(1,self.embeding_shape),dtype='float32'),
                        trainable=True
                     )
#             embed = tf.Variable(lambda:tf.random.truncated_normal(shape=(1,self.embeding_shape), stddev=0.01))
            self.embeds.append(embed)
            
    def call(self,x_dense):
        dense_kd_embed = []
        for i,_input in enumerate(x_dense):
#             f = _input.name[:4]
#             embed = tf.Variable(lambda:tf.random.truncated_normal(shape=(1,self.embeding_shape), stddev=0.01,name='dense'+str(i) ))
            
            scaled_embed = tf.expand_dims(_input*self.embeds[i], axis=1)
            dense_kd_embed.append(scaled_embed)
        return dense_kd_embed
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
#             'sparse_feat_config': self.sparse_feat_config,
            'embeding_shape': self.embeding_shape,
        })
        return config
# t = Embedding_dense(sparse_feat_config,1)
# y = t(inputs_sparse)    

In [24]:
# attention 层
class attention_layer(tf.keras.layers.Layer):
    def __init__(self,d,n_attention_head):
        super(attention_layer,self).__init__()
        self.n_attention_head = n_attention_head
#         self.embed_map = embed_map
        self.dense_q = []
        self.dense_k = []
        self.dense_v = []  
        self.attention_layers = []
        for i in range(self.n_attention_head):
#             print('init',i)
            # (self.embed_map)
            self.dense_q.append(layer.Dense(d))
            self.dense_k.append(layer.Dense(d))
            self.dense_v.append(layer.Dense(d))
            self.attention_layers.append(layer.Attention())    
    def call(self, embed_map):
       # embed_map shape (None, 39, 8),
        attention_heads = []
#         print('ggggg',n)
        for i in range(self.n_attention_head):
#             print('i',i)
            attention_output = self.attention_layers[i]([self.dense_q[i](embed_map), self.dense_k[i](embed_map), self.dense_k[i](embed_map)])
            attention_heads.append(attention_output)
        if self.n_attention_head > 1:
            muti_attention_output = layer.Concatenate(axis=-1)(attention_heads)
        else:
            muti_attention_output = attention_output
        return muti_attention_output
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_attention_head': self.n_attention_head,
            'd':self.d
        })
        return config  

In [25]:
class AutoInt:
    def __init__(self,sparse_feat_config,dense_feats):
        
        self.sparse_feat_config= sparse_feat_config
        self.inputs_sparse, self.inputs_dense = self.build_input(sparse_feat_config,dense_feats)
        
#         self.firsr_cross_dense = firsr_cross_dense(self.sparse_feat_config)
        self.Embedding_sparse_layer = Embedding_sparse_layer(self.sparse_feat_config,8)
        self.Embedding_dense_layer = Embedding_dense_layer(8)
        # 通过循环建立多个交互层
        self.attention_layers = []
        for i in range(2):
            self.attention_layers.append(attention_layer(6,3))  # 第一个参数表示映射维度，第二个表示头的个数
        
#         self.DNN = DNN([128,128,64])
        
        self.AutoInt =  self.build_model() 
    def build_input(self,sparse_feat_config,dense_feats):
        inputs_sparse = []
        inputs_dense = []
        for key in sparse_feat_config:
            inputs_sparse.append(layer.Input(shape=(1,),name=key))
        for key in dense_feats:
            inputs_dense.append(layer.Input(shape=(1,),name=key))
        
        return inputs_sparse, inputs_dense

    def build_model(self, num_lays = 3):
        # DCN输入部分
        # sparse特征嵌入
        sparse_embed_lookup =  self.Embedding_sparse_layer(self.inputs_sparse) 
        dense_embed_lookup =  self.Embedding_dense_layer(self.inputs_dense) 
        input_embeds = sparse_embed_lookup + dense_embed_lookup
        embed_map = layer.Concatenate(axis=1)(input_embeds) 
        
        x_l = embed_map
        for attention_layer in self.attention_layers:
              x_l = attention_layer(x_l)
        autoint_layer = layer.Flatten()(x_l)
        output_layer = layer.Dense(1, activation="sigmoid")(autoint_layer)
#         return output_layer
        # 初始化模型
        model = Model(self.inputs_sparse + self.inputs_dense, outputs=output_layer)
        model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                      loss= 'binary_crossentropy',
                      metrics=['AUC'])
        return model
    

    def train(self,train_data,train_label,valid_data, valid_label,batch_size,epochs,callbacks):
        self.AutoInt.fit(train_data,train_label,
                  batch_size=batch_size, epochs=epochs, verbose=1, 
                  validation_data=(valid_data, valid_label),
                  callbacks = callbacks
                 )        
        
# AutoInt(sparse_feat_config, dense_feats).build_model()

In [20]:
# 五折交叉 + 提前停止 + 保存模型
# tf.compat.v1.disable_eager_execution() 加入会报错

skf = StratifiedKFold(n_splits = 5, random_state=1996, shuffle = True)
for idx, (train_idx, val_idx) in enumerate(skf.split(total_data,total_data['label'])):
    print('fold:',idx)
    K.clear_session()
    train_sparse_x,train_dense_x,train_label = make_data(total_data,train_idx)
    val_sparse_x,val_dense_x,val_label = make_data(total_data,val_idx) 
    # 定义回调
    
    # 保存模型
    file_path = f'./model/{idx}.h5'

    checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True, mode='min')
    # metric 不提高时，减小学习率
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
    # val_loss 连续两次提升小于 1e-2，提前停止
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2,verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]

    # 初始化模型
    AutoInt_ctr = AutoInt(sparse_feat_config, dense_feats)
    AutoInt_ctr.train(train_sparse_x+train_dense_x,train_label,
                 val_sparse_x+val_dense_x,val_label,
                12800,1, callbacks=callbacks)    

fold: 0
input [TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1])]


/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


38/38 [==============================] - ETA: 0s - loss: 146.9688 - auc: 0.5422
Epoch 00001: val_loss improved from inf to 11.01196, saving model to ./model/0.h5
38/38 [==============================] - 31s 807ms/step - loss: 146.9688 - auc: 0.5422 - val_loss: 11.0120 - val_auc: 0.6388 - lr: 0.0010
fold: 1
input [TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1]), TensorShape([None, 1])]
38/38 [==============================] - ETA: 0s - loss: 146.8706 - auc: 0.5253
Epoch 00001: val_loss improved from inf to 11.00314, saving model to ./model/1.h5
38/38 [==============================] - 30s 796ms/step - loss: 146.8706 - auc: 0.5253 - val_loss: 11.0031 - val_auc: 0.5952 - lr: 0.0010
fold: 2
input [TensorShape([None, 1]), TensorShape([None, 1]), TensorShap

KeyboardInterrupt: 

In [26]:
# 模型的加载与预测
AutoInt_ctr = AutoInt(sparse_feat_config, dense_feats).build_model()
AutoInt_ctr.load_weights('./model/0.h5')
AutoInt_ctr.predict(val_sparse_x+val_dense_x)

array([[0.2748378 ],
       [0.2906196 ],
       [0.27955478],
       ...,
       [0.28339756],
       [0.26656428],
       [0.25270134]], dtype=float32)

In [27]:
AutoInt_ctr.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C1 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C2 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C3 (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
C4 (InputLayer)                 [(None, 1)]          0                                            
____________________________________________________________________________________________